# Complete Workflow for generating ATS input for Coweeta: Part 1 - Surface Meshing
 
This workflow demonstrates how to develop a simulation campaign for integrated hydrology using ATS. 
 
### Overview
 
Part 1 of this workflow focuses on creating a terrain-following surface mesh. It uses arbitrary polygonal shapes to generate computational elements that align with key watershed features. For instance, elongated quadrilaterals and pentagons at junctions are aligned with stream centerlines to efficiently resolve stream networks.
 
The notebook processes hydrography data to meet tessellation requirements and performs tessellation under specific constraints. It also incorporates hydrologic conditioning of the surface topography within the mesh. The resulting surface mesh is stored as an `m2` class object and saved as a pickle file for use in Part 2.

### Datasets Used
 
- `NHD Plus`: Hydrography
- `3DEP`: Elevation


In [ ]:
# setting up logging first or else it gets preempted by another package
import watershed_workflow.ui
watershed_workflow.ui.setup_logging(1)

In [ ]:
import os,sys
import numpy as np
from matplotlib import pyplot as plt
import shapely
import pandas as pd

import watershed_workflow 
import watershed_workflow.config
import watershed_workflow.sources
import watershed_workflow.utils
import watershed_workflow.plot
import watershed_workflow.mesh
import watershed_workflow.regions
import watershed_workflow.sources.standard_names as names


# set the default figure size for notebooks
plt.rcParams["figure.figsize"] = (8, 6)


# Input: Parameters and other source data

Note, this section will need to be modified for other runs of this workflow in other regions.

In [ ]:
# Force Watershed Workflow to pull data from this directory rather than a shared data directory.
# This picks up the Coweeta-specific datasets set up here to avoid large file downloads for 
# demonstration purposes.
#
def splitPathFull(path):
    """
    Splits an absolute path into a list of components such that
    os.path.join(*splitPathFull(path)) == path
    """
    parts = []
    while True:
        head, tail = os.path.split(path)
        if head == path:  # root on Unix or drive letter with backslash on Windows (e.g., C:\)
            parts.insert(0, head)
            break
        elif tail == path:  # just a single file or directory
            parts.insert(0, tail)
            break
        else:
            parts.insert(0, tail)
            path = head
    return parts

cwd = splitPathFull(os.getcwd())

# Note, this directory is where downloaded data will be put as well
data_dir = os.path.join(*(cwd + ['input_data',]))
def toInput(filename):
    return os.path.join(data_dir, filename)

output_dir = os.path.join(*(cwd + ['output_data',]))
def toOutput(filename):
    return os.path.join(output_dir, filename)

work_dir = os.path.join(*cwd)
def toWorkingDir(filename):
    return os.path.join(work_dir, filename)
       

In [ ]:
# Set the data directory to the local space to get the locally downloaded files
# REMOVE THIS CELL for general use outside fo Coweeta
watershed_workflow.config.setDataDirectory(data_dir)


In [ ]:
## Parameters cell -- this provides all parameters that can be changed via pipelining to generate a new watershed. 
name = 'Coweeta'
coweeta_shapefile = os.path.join('input_data', 'coweeta_basin.shp')

# Geometric parameters
# -- parameters to clean and reduce the river network prior to meshing
simplify = 60                   # length scale to target average edge 
ignore_small_rivers = 2         # remove rivers with fewer than this number of reaches -- important for NHDPlus HR 
prune_by_area_fraction = 0.01   # prune any reaches whose contributing area is less than this fraction of the domain

# -- mesh triangle refinement control
refine_d0 = 200
refine_d1 = 600

refine_L0 = 125
refine_L1 = 300

refine_A0 = refine_L0**2 / 2
refine_A1 = refine_L1**2 / 2


In [ ]:
# a dictionary of pickle filenames -- will include all pickle files generated
pickle_filenames = {}

In [ ]:
# Note that, by default, we tend to work in the DayMet CRS because this allows us to avoid
# reprojecting meteorological forcing datasets.
crs = watershed_workflow.crs.daymet_crs

In [ ]:
# get the shape and crs of the shape
coweeta_mgr = watershed_workflow.sources.ManagerShapefile(coweeta_shapefile)
coweeta = coweeta_mgr.getShapes(out_crs=crs)
coweeta.rename(columns={'AREA' : names.AREA, 'LABEL' : names.NAME}, inplace=True)
coweeta[names.ID] = coweeta.index.values
coweeta.set_index(names.ID, inplace=True, drop=True)

In [ ]:
# print(type(coweeta)) 
# display(coweeta)

In [ ]:
# set up a dictionary of source objects
#
# Data sources, also called managers, deal with downloading and parsing data files from a variety of online APIs.
sources = watershed_workflow.sources.getDefaultSources()
sources['hydrography'] = watershed_workflow.sources.hydrography_sources['NHDPlus HR']

#
# This demo uses a few datasets that have been clipped out of larger, national
# datasets and are distributed with the code.  This is simply to save download
# time for this simple problem and to lower the barrier for trying out
# Watershed Workflow.  A more typical workflow would delete these lines (as 
# these files would not exist for other watersheds).
#
# The default versions of these download large raster and shapefile files that
# are defined over a very large region (globally or the entire US).
#
# DELETE THIS SECTION for non-Coweeta runs
dtb_file = os.path.join(data_dir, 'DTB', 'DTB.tif')
geo_file = os.path.join(data_dir, 'GLHYMPS', 'GLHYMPS.shp')

# GLHYMPs is a several-GB download, so we have sliced it and included the slice here
sources['geologic structure'] = watershed_workflow.sources.ManagerGLHYMPS(geo_file)

# The Pelletier DTB map is not particularly accurate at Coweeta -- the SoilGrids map seems to be better.
# Here we will use a clipped version of that map.
sources['depth to bedrock'] = watershed_workflow.sources.ManagerRaster(dtb_file)

# END DELETE THIS SECTION

# log the sources that will be used here
watershed_workflow.sources.logSources(sources)


# Basin Geometry

In this section, we choose the basin, the streams to be included in the stream-aligned mesh, and make sure that all are resolved discretely at appropriate length scales for this work.

## the Watershed

In [ ]:
# Construct and plot the WW object used for storing watersheds
watershed = watershed_workflow.split_hucs.SplitHUCs(coweeta)
watershed.plot()

In [ ]:
# ** Training cell **
# # methods and attributes
# for f in dir(watershed):
#    print(f)

In [ ]:
# ** Training cell **
# watershed
# watershed.df
# watershed.exterior
# watershed.linestrings
   

DN: not sure if it makes sense to go over handled collection since we have only a single watershed

## the Rivers 

** Training Note **
<b> Construction </b>
Rivers are constructed using a collection of reaches using one of the following methods:

1) `geometry` looks at coincident coordinates. This is needed when working with non-NHD data.
2) `hydroseq` is valid only for NHDPlus data. This method uses the NHDPlus VAA tables Hydrologic Sequence. 
3) `native` reads a natively dumped list of rivers.

In [ ]:
# download/collect the river network within that shape's bounds
reaches = sources['hydrography'].getShapesByGeometry(watershed.exterior, crs, out_crs=crs)
rivers = watershed_workflow.river_tree.createRivers(reaches, method='hydroseq') # other method: 'geometry'
watershed_orig, rivers_orig = watershed, rivers

In [ ]:
# ** Training cell **
#reaches 
#reaches.iloc[0]
#dict(reaches.iloc[0])
#reaches.iloc[0].geometry


In [ ]:
%matplotlib ipympl

In [ ]:
# plot the rivers and watershed
def plot(ws, rivs, ax=None):
    if ax is None:
        fig, ax = plt.subplots(1, 1)
    ws.plot(color='k', marker='+', markersize=10, ax=ax)
    for river in rivs:
        river.plot(marker='x', markersize=10, ax=ax)

plot(watershed, rivers)

### ----- Training Cells: River Tree Features -----

In [ ]:
# ** Training cell **
river = rivers[0]
# river.df

In [ ]:
# Nodes
# A `River` is a composed of nodes. A single node in the River is also a `River` object, representing one reach and its upstream children.

# Traversing tree
# Basic tree traversal uses the `preOrder()` method, which is a depth-first traversal of the tree.
nodes = [node for node in river.preOrder()]
for node in nodes:
    print(node.properties['ID'])

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
river.plot(ax=ax, color='b')
# node = nodes[3]
# node.plot(ax=ax, color='r', linestyle='-')  ##### CHANGE TO 2, 3, .. and march up the river #####
# ax.plot(*node.linestring.xy, color='g', linestyle='--')
ax.set_aspect('equal')


In [ ]:
# ** Training cell ** N
# Parent/Child Relationships
node = river # nodes[3]

fig, ax = plt.subplots(figsize=(5, 5))
node.plot(ax=ax, color='b', linestyle='-')
node.children[0].plot(ax=ax, color='r', linestyle='-')
node.children[1].plot(ax=ax, color='g', linestyle='-')
# ax.plot(*node.linestring.xy, color='g', linestyle='--')
ax.set_aspect('equal')


In [ ]:
# we can access siblings 
node = nodes[1]

siblings = list(node.siblings); print(siblings)
fig, ax = plt.subplots(1, 1, figsize=(3,3))
node.plot(color='b', ax=ax)
siblings[0].plot(ax=ax, color='r')



In [ ]:
# we can access parent node
parent = node.parent
fig, ax = plt.subplots(1, 1, figsize=(3,3))
node.plot(color='b', ax=ax)
ax.plot(*parent.linestring.xy, color='r', linestyle='-')

In [ ]:
# methods 
node.linestring.length

rivers[0].linestring.length

### ------------------------

In [ ]:
# keeping the originals for plotting comparisons
def createCopy(watershed, rivers):
    """To compare before/after, we often want to create copies.  Note in real workflows most things are done in-place without copies."""
    return watershed.deepcopy(), [r.deepcopy() for r in rivers]
    

In [ ]:
# ** Training cell ** 
# refinement controls
refine_d0 = 200
refine_d1 = 600

refine_L0 = 125
refine_L1 = 300

Looking at the river resampling strategies in the code, here's a concise summary:

The watershed workflow provides multiple strategies for resampling river networks, allowing users to control the resolution and detail of river representations based on different criteria and requirements.

• **Fixed Length**: Uses a single uniform target segment length across all river reaches for consistent resolution

• **Property-Based**: Reads target lengths from each reach's stored properties, enabling different resolutions for different river segments

• **Function-Driven**: Employs custom functions to dynamically compute target lengths with min/max bounds, offering maximum flexibility for complex logic

• **Distance-Adaptive**: Varies resolution based on proximity to reference shapes, providing finer detail near important features and coarser detail elsewhere

In [ ]:
watershed, rivers = createCopy(watershed_orig, rivers_orig)

# simplifying -- this sets the discrete length scale of both the watershed boundary and the rivers
watershed_workflow.simplify(watershed, rivers, refine_L0, refine_L1, refine_d0, refine_d1)

# simplify may remove reaches from the rivers object
# -- this call removes any reaches from the dataframe as well, signaling we are all done removing reaches
#
# ETC: NOTE -- can this be moved into the simplify call?
for river in rivers:
    river.resetDataFrame()

# Now that the river network is set, find the watershed boundary outlets
for river in rivers:
    watershed_workflow.hydrography.findOutletsByCrossings(watershed, river)

In [ ]:
plot(watershed, rivers)

In [ ]:
# this generates a zoomable map, showing different reaches and watersheds, 
# with discrete points.  Problem areas are clickable to get IDs for manual
# modifications.
m = watershed.explore(marker=False)
for river in rivers_orig:
    m = river.explore(m=m, column=None, color='black', name=river['name']+' raw', marker=False)
for river in rivers:
    m = river.explore(m=m)
    
m = watershed_workflow.makeMap(m)
m

## Mesh Geometry

Now we create stream-aligned mesh conforming to the above discretization of river tree and watershed boundary. River width can be provided either using a dictionary stream-order:width, or width can be assigned as a property for each node, which can be read on the fly while assigning width of quad elements in the river mesh. 

In [ ]:
# Refine triangles if they get too acute
min_angle = 32 # degrees

# width of reach by stream order (order:width)
widths = dict({1:8,2:12,3:16})

# create the mesh
m2, areas, dists = watershed_workflow.tessalateRiverAligned(watershed, rivers, 
                                                            river_width=widths,
                                                            refine_min_angle=min_angle,
                                                            refine_distance=[refine_d0, refine_A0, refine_d1, refine_A1],
                                                            diagnostics=True)

### ----- Training Cells: Tessellation Options --------

<b> Internal Boundaries and Refinement </b>

In [ ]:
refine_d0 = 200
refine_d1 = 600

refine_A0 = refine_L0**2 / 2
refine_A1 = refine_L1**2 / 2

In [ ]:
from shapely.geometry import box
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as mplPolygon

# Create a box shape using shapely's box function
box_shape = box(1445315, -646934, 1445805, -646458)

# Set up the plot
fig, ax = plt.subplots(figsize=(2.5,2.5))

# Plot the box shape using ax.add_patch
box_patch = mplPolygon(list(box_shape.exterior.coords), closed=True, edgecolor='r', linewidth=1, alpha=0.7, facecolor='none')
ax.add_patch(box_patch)

# Plot the watershed and rivers
watershed.plot(ax=ax, alpha=0.5, color='k', linewidth=1)
rivers[0].plot(ax=ax, color='b', linewidth=0.5)

# Display the plot
plt.show()

In [ ]:
# Refine triangles if they get too acute
min_angle = 32 # degrees

# width of reach by stream order (order:width)
widths = dict({1:8,2:12,3:16})

# create the mesh
m2, areas, dists = watershed_workflow.tessalateRiverAligned(watershed, rivers, 
                                                            river_width=widths,
                                                            refine_min_angle=min_angle,
                                                            # refine_polygons = [[box_shape], 500],
                                                            # internal_boundaries = [box_shape],
                                                            refine_distance=[refine_d0, refine_A0, refine_d1, refine_A1],
                                                            diagnostics=True)

In [ ]:
# plotting surface mesh with elevations
fig, ax = plt.subplots(figsize=(5, 5))

# Plot the main mesh without a color bar
mp = m2.plot(facecolor='darkgray', edgecolor='w', ax=ax, linewidth=0.5, colorbar=False)
ax.set_title('Surface Mesh')
ax.set_aspect('equal', 'datalim')
watershed.plot(ax=ax, alpha=0.5, color='k', linewidth=1)

# rivers[0].plot(ax=ax, color='red', linewidth=0.5)

plt.show()


### ----------------------------

In [ ]:
# get a raster for the elevation map, based on 3DEP
dem = sources['DEM'].getDataset(watershed.exterior.buffer(100), watershed.crs)['dem']

# provide surface mesh elevations
watershed_workflow.elevate(m2, dem)

In [ ]:
# Plot the DEM raster
fig, ax = plt.subplots()

# Plot the DEM data
im = dem.plot(ax=ax, cmap='terrain', add_colorbar=False)

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Elevation (m)', rotation=270, labelpad=15)

# Add title and labels
ax.set_title('Digital Elevation Model (DEM)', fontsize=14, fontweight='bold')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')

# Set equal aspect ratio
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

In the pit-filling algorithm, we want to make sure that river corridor is not filled up. Hence we exclude river corridor cells from the pit-filling algorithm.

In [ ]:
# hydrologically condition the mesh, removing pits
river_mask=np.zeros((len(m2.conn)))
for i, elem in enumerate(m2.conn):
    if not len(elem)==3:
        river_mask[i]=1     
watershed_workflow.condition.fillPitsDual(m2, is_waterbody=river_mask)

There are a range of options to condition river corridor mesh. We hydrologically condition the river mesh, ensuring unimpeded water flow in river corridors by globally adjusting flowlines to rectify artificial obstructions from inconsistent DEM elevations or misalignments. Please read the documentation for more information


In [ ]:
# conditioning river mesh
#
# adding elevations to the river tree for stream bed conditioning
watershed_workflow.condition.setProfileByDEM(rivers, dem)

# conditioning the river mesh using NHD elevations
watershed_workflow.condition.conditionRiverMesh(m2, rivers[0])

In [ ]:
# plotting surface mesh with elevations
fig, ax = plt.subplots()
ax2 = ax.inset_axes([0.65,0.05,0.3,0.5])
cbax = fig.add_axes([0.05,0.05,0.9,0.05])

mp = m2.plot(facecolors='elevation', edgecolors=None, ax=ax, linewidth=0.5, colorbar=False)
cbar = fig.colorbar(mp, orientation="horizontal", cax=cbax)
ax.set_title('surface mesh with elevations')
ax.set_aspect('equal', 'datalim')

mp2 = m2.plot(facecolors='elevation', edgecolors='white', ax=ax2, colorbar=False)
ax2.set_aspect('equal', 'datalim')

xlim = (1.4433e6, 1.4438e6)
ylim = (-647000, -647500)

ax2.set_xlim(xlim)
ax2.set_ylim(ylim)
ax2.set_xticks([])
ax2.set_yticks([])

ax.indicate_inset_zoom(ax2, edgecolor='k')

cbar.ax.set_title('elevation [m]')

plt.show()


## River/Stream-specific LabeledSets

In [ ]:
# add labeled sets for subcatchments and outlets
watershed_workflow.regions.addWatershedAndOutletRegions(m2, watershed, outlet_width=250, exterior_outlet=True)

# add labeled sets for river corridor cells
watershed_workflow.regions.addRiverCorridorRegions(m2, rivers)

# add labeled sets for river corridor cells by order
watershed_workflow.regions.addStreamOrderRegions(m2, rivers)

In [ ]:
for ls in m2.labeled_sets:
    print(f'{ls.setid} : {ls.entity} : {len(ls.ent_ids)} : "{ls.name}"')

<b> This concludes part 1 of the workflow, in which spatial discretization of the watershed and river network is performed, and surface mesh geometry is created. </b>

In [ ]:
import pickle
import os

intermediate_dir = './intermediate_files/'

# Ensure the intermediate directory exists
os.makedirs(intermediate_dir, exist_ok=True)

# Save m2 and watershed objects using pickle
with open(os.path.join(intermediate_dir, 'm2.pkl'), 'wb') as f:
    pickle.dump(m2, f)

with open(os.path.join(intermediate_dir, 'watershed.pkl'), 'wb') as f:
    pickle.dump(watershed, f)

# Concatenate river dataframes and save as parquet
river_df = pd.concat([river.to_dataframe() for river in rivers])
river_df.to_parquet(os.path.join(intermediate_dir, 'rivers.parquet'))

print("Intermediate files saved successfully.")